In [1]:
from pydrake.all import (
    DiagramBuilder,
    Simulator,
    StartMeshcat,
    InverseDynamicsController,
    MeshcatPoseSliders,
    Context,
    RigidTransform,
    RotationMatrix,
    Sphere,
    Rgba,
    LeafSystem,
    RollPitchYaw,
    MultibodyPlant,
    Parser,
    Solve,
    SolutionResult,
    AddMultibodyPlantSceneGraph,
    AddFrameTriadIllustration,
    FixedOffsetFrame,
    TrajectorySource,
    Trajectory,
    PiecewisePose,
    PiecewisePolynomial,
    ConstantValueSource,
    AbstractValue
)
from pydrake.multibody import inverse_kinematics
from pydrake.multibody.inverse_kinematics import (
    DifferentialInverseKinematicsParameters,
    DifferentialInverseKinematicsIntegrator,
)
from pydrake.trajectories import PiecewisePolynomial
from pydrake.systems.primitives import LogVectorOutput
from pydrake.systems.framework import BasicVector

from manipulation.station import (
    LoadScenario,
    MakeHardwareStation,
)
from manipulation.utils import RenderDiagram, running_as_notebook
from manipulation.meshcat_utils import AddMeshcatTriad
from pydrake.multibody.plant import ContactResults
from pydrake.multibody.tree import JacobianWrtVariable

import random
import numpy as np

In [2]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7000


Click the link above to open Meshcat in your browser!


In [ ]:
class RestingMoleController(LeafSystem):
    """
    Keeps all moles at a fixed position.
    For rest_height = 0.0, the mole base sits flush with the socket.
    """

    def __init__(self, plant, mole_joints, rest_height=0.):
        super().__init__()

        self.plant = plant
        self.mole_joints = mole_joints
        self.rest_height = rest_height   # height for ALL moles (usually 0.0)

        # Input: full plant state [q; v]
        self.state_in = self.DeclareVectorInputPort(
            "x", plant.num_multibody_states()
        )

        # One actuation output per mole
        self.mole_out = {}
        for ij, joint in mole_joints.items():
            self.mole_out[ij] = self.DeclareVectorOutputPort(
                f"mole_{ij[0]}_{ij[1]}_u", 1,
                (lambda ctx, out, ij=ij: self._CalcMoleControl(ctx, out, ij))
            )

    def _CalcMoleControl(self, context, output, ij):
        # Desired height is constant for all moles
        target = self.rest_height

        # Extract plant state
        x = self.state_in.Eval(context)
        nq = self.plant.num_positions()
        q = x[:nq]
        v = x[nq:]

        joint = self.mole_joints[ij]
        qj = q[joint.position_start()]
        vj = v[joint.velocity_start()]

        # PD control
        kp = 600.0
        kd = 12.0
        u = kp * (target - qj) - kd * vj

        output.SetFromVector([u])
    

class PoppingMoleController(LeafSystem):
    """
    A controller that keeps all moles resting at joint height = 0.0 (base touching socket)
    and repeatedly selects ONE mole at random to rise to a commanded height.
    """

    def __init__(self, plant, mole_joints,
                 rise_height=0.09,
                 rest_height=0.0,
                 min_up=1.5, max_up=1.5):
        super().__init__()

        self.plant = plant
        self.mole_joints = mole_joints
        self.rise_height = rise_height
        self.rest_height = rest_height

        self.min_up = min_up
        self.max_up = max_up

        # ---------------------------
        # State machine
        # ---------------------------
        # Pick the first active mole
        self.active = random.choice(list(mole_joints.keys()))
        # Set the time when we will switch to the next mole
        self.next_switch_time = random.uniform(min_up, max_up)

        # TODO: Track whether each mole is still allowed to pop up
        self.mole_alive_idx = self.DeclareAbstractState(
            AbstractValue.Make({ij: True for ij in self.mole_joints})
        )

        # Next time to switch the active mole
        self.next_switch_idx = self.DeclareAbstractState(
            AbstractValue.Make(random.uniform(min_up, max_up))
        )

        # Input: full plant state [q; v]
        self.state_in = self.DeclareVectorInputPort(
            "x", plant.num_multibody_states()
        )

        # Output: one actuation per mole
        self.mole_out = {}
        for ij, joint in mole_joints.items():
            self.mole_out[ij] = self.DeclareVectorOutputPort(
                f"mole_{ij[0]}_{ij[1]}_u", 1,
                (lambda ctx, out, ij=ij: self._CalcMoleControl(ctx, out, ij))
            )

    # --------------------------------------------------------------
    # Helper: pick a new mole different from the current one
    # --------------------------------------------------------------
    def _choose_new_mole(self):
        keys = list(self.mole_joints.keys())
        keys.remove(self.active)
        return random.choice(keys)

    # --------------------------------------------------------------
    # Main PD control calculation
    # --------------------------------------------------------------
    def _CalcMoleControl(self, context, output, ij):

        t = context.get_time()

        # --------------------------------------------
        # Random state switching: one mole at a time
        # --------------------------------------------
        if len(self.mole_joints) > 1:
            if t >= self.next_switch_time and ij == self.active:
                self.active = self._choose_new_mole()
                self.next_switch_time = t + random.uniform(self.min_up, self.max_up)

        # --------------------------------------------
        # Target height:
        #   active mole → rise_height
        #   others      → rest_height (0.0)
        # --------------------------------------------
        target = self.rise_height if ij == self.active else self.rest_height

        # --------------------------------------------
        # Extract q and v for this mole
        # --------------------------------------------
        x = self.state_in.Eval(context)
        nq = self.plant.num_positions()
        q = x[:nq]
        v = x[nq:]

        joint = self.mole_joints[ij]
        qj = q[joint.position_start()]
        vj = v[joint.velocity_start()]

        # --------------------------------------------
        # PD control
        # --------------------------------------------
        kp = 600.0
        kd = 12.0
        u = kp * (target - qj) - kd * vj

        output.SetFromVector([u])

In [51]:
HAMMER_SDF_PATH = "/workspaces/whackamole/bonkbot/sim/assets/hammer.sdf"
GRID_SDF_PATH = "/workspaces/whackamole/bonkbot/sim/assets/grid_board.sdf"
MOLE_SDF_PATH = "/workspaces/whackamole/bonkbot/sim/assets/mole.sdf"

scenario_string = f"""directives:

# ===============================================================
# IIWA
# ===============================================================
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/sdf/iiwa7_with_box_collision.sdf
    default_joint_positions:
      iiwa_joint_1: [-1.57]
      iiwa_joint_2: [0.1]
      iiwa_joint_3: [0]
      iiwa_joint_4: [-1.2]
      iiwa_joint_5: [0]
      iiwa_joint_6: [1.6]
      iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0

# ===============================================================
# Hammer
# ===============================================================
- add_model:
    name: hammer
    file: file://{HAMMER_SDF_PATH}
    default_free_body_pose:
      hammer_link:
        translation: [0, 0, 0]
        rotation: !Rpy {{ deg: [0, 0, 0] }}

- add_weld:
    parent: iiwa::iiwa_link_7
    child: hammer::hammer_link
    X_PC:
      translation: [0, 0, 0.06]
      rotation: !Rpy {{deg: [0, -90, 0] }}

# ===============================================================
# Grid Board + Moles
# ===============================================================
- add_model:
    name: grid_board
    file: file://{GRID_SDF_PATH}

- add_weld:
    parent: world
    child: grid_board::board
    X_PC:
      translation: [0, -0.75, 0]
      rotation: !Rpy {{ deg: [0, 0, 0] }}

# 3x3 mole grid
- add_model:
    name: mole_0_0
    file: file://{MOLE_SDF_PATH}
- add_weld:
    parent: grid_board::board
    child: mole_0_0::socket
    X_PC: {{translation: [-0.2, -0.2, 0.0125]}}

- add_model:
    name: mole_0_1
    file: file://{MOLE_SDF_PATH}
- add_weld:
    parent: grid_board::board
    child: mole_0_1::socket
    X_PC: {{translation: [0.0, -0.2, 0.0125]}}

- add_model:
    name: mole_0_2
    file: file://{MOLE_SDF_PATH}
- add_weld:
    parent: grid_board::board
    child: mole_0_2::socket
    X_PC: {{translation: [0.2, -0.2, 0.0125]}}

- add_model:
    name: mole_1_0
    file: file://{MOLE_SDF_PATH}
- add_weld:
    parent: grid_board::board
    child: mole_1_0::socket
    X_PC: {{translation: [-0.2, 0.0, 0.0125]}}

- add_model:
    name: mole_1_1
    file: file://{MOLE_SDF_PATH}
- add_weld:
    parent: grid_board::board
    child: mole_1_1::socket
    X_PC: {{translation: [0.0, 0.0, 0.0125]}}

- add_model:
    name: mole_1_2
    file: file://{MOLE_SDF_PATH}
- add_weld:
    parent: grid_board::board
    child: mole_1_2::socket
    X_PC: {{translation: [0.2, 0.0, 0.0125]}}

- add_model:
    name: mole_2_0
    file: file://{MOLE_SDF_PATH}
- add_weld:
    parent: grid_board::board
    child: mole_2_0::socket
    X_PC: {{translation: [-0.2, 0.2, 0.0125]}}

- add_model:
    name: mole_2_1
    file: file://{MOLE_SDF_PATH}
- add_weld:
    parent: grid_board::board
    child: mole_2_1::socket
    X_PC: {{translation: [0.0, 0.2, 0.0125]}}

- add_model:
    name: mole_2_2
    file: file://{MOLE_SDF_PATH}
- add_weld:
    parent: grid_board::board
    child: mole_2_2::socket
    X_PC: {{translation: [0.2, 0.2, 0.0125]}}

model_drivers:
    iiwa: !IiwaDriver
      control_mode: position_only
"""

In [52]:
def get_prehit_pose(X_WO: RigidTransform, X_HL7: RigidTransform) -> tuple[RigidTransform, RigidTransform]:
    """
    Given the object pose X_WO, compute a prehit pose in hammer face frame and iiwa link 7 frame.
    
    Parameters:
        X_WO: pose of target object in world frame
        X_HL7: pose of iiwa link 7 in hammer face frame
    """
    p_OH = np.array([0.0, -0.2, 0.0])
    R_OH = RotationMatrix.MakeYRotation(-np.pi/2) @ RotationMatrix.MakeXRotation(-np.pi/2)
    X_OH = RigidTransform(R_OH, p_OH) # pose of hammer face from object frame
    X_WH_prehit = X_WO @ X_OH # prehit pose of hammer face 
    X_WL7_prehit = X_WH_prehit @ X_HL7
    return X_WH_prehit, X_WL7_prehit


def get_mole_prehit_pose(X_WO: RigidTransform, X_HL7: RigidTransform):
    """
    Computes a prehit hammer pose over an object (mole).
    Align hammer z-axis downward, keep yaw fixed, position above mole.
    """
    # --- Desired prehit offset relative to mole center ---
    p_OH = np.array([0.0, 0.0, 0.12])   # 12 cm above mole, tune as needed
    
    # --- Build a stable orientation ---
    # z-axis: straight down in world
    z = np.array([0, 0, -1])

    # x-axis: choose a fixed world direction
    x = np.array([1, 0, 0])

    # Orthonormalize
    x = x / np.linalg.norm(x)
    y = np.cross(z, x)

    R_WH = RotationMatrix(np.column_stack((x, y, z)))

    # --- Build world-frame hammer pose ---
    p_WH = X_WO.translation() + p_OH
    X_WH_prehit = RigidTransform(R_WH, p_WH)

    # --- Corresponding iiwa link 7 pose ---
    X_WL7_prehit = X_WH_prehit @ X_HL7

    return X_WH_prehit, X_WL7_prehit


def solve_ik(plant, plant_context, X_WH, q_guess=None, pos_tol=1e-3, theta_bound=1e-2):
    # Get the hammer face frame 
    hammer = plant.GetModelInstanceByName("hammer")
    hammer_face_frame = plant.GetFrameByName("hammer_face",hammer)
    W = plant.world_frame()

    # Moles introduce extra DOFs
    iiwa = plant.GetModelInstanceByName("iiwa")

    # If user did not provide a guess, use iiwa’s current positions
    if q_guess is None:
        q_guess = plant.GetPositions(plant_context, iiwa)

    full_q_guess = plant.GetPositions(plant_context).copy()
    full_q_guess[0 : 7] = q_guess

    # Set up IK problem
    ik = inverse_kinematics.InverseKinematics(plant, plant_context)

    # Extract desired position/orientation from X_WG
    R_WH_desired = X_WH.rotation()
    p_WH_desired = X_WH.translation()

    # Position constraint: place origin of hammer face at p_WH_desired (within a small box)
    # Offset the hammer so it stays above the target (e.g., top of a mole)
    strike_offset = np.array([0, 0, 0])
    
    ik.AddPositionConstraint(
        frameA=W,
        p_BQ=np.zeros(3),        # target point in world
        frameB=hammer_face_frame,
        p_AQ_lower=(p_WH_desired + strike_offset) - pos_tol,
        p_AQ_upper=(p_WH_desired + strike_offset) + pos_tol,
    )

    # Orientation constraint: align world and hammer_face_frame up to some tolerance
    ik.AddOrientationConstraint(
        frameAbar=W,
        R_AbarA=RotationMatrix(),      # identity
        frameBbar=hammer_face_frame,
        R_BbarB=R_WH_desired,
        theta_bound=theta_bound,              # rad
    )

    # Add collision constraint to ensure collision free pre-hit pose
    # ik.AddMinimumDistanceConstraint(0.02)

    prog = ik.prog()
    prog.SetInitialGuess(ik.q(), full_q_guess)
    result = Solve(prog)

    if result.get_solution_result() != SolutionResult.kSolutionFound:
        return result.GetSolution(ik.q()), False
    return result.GetSolution(ik.q()), True


def make_joint_space_position_trajectory(path, timestep=1.0):
    traj = None
    times = [timestep * i for i in range(len(path))]
    Q = np.column_stack(path)
    traj = PiecewisePolynomial.FirstOrderHold(times, Q)
    return traj

In [53]:
class HammerContactForce(LeafSystem):
    """
    Reads ContactResults from the plant and outputs the scalar contact force
    on the hammer body along n_hat (in world frame).
    """
    def __init__(self, plant, hammer_body_index, n_hat):
        super().__init__()
        self._plant = plant
        self._hammer_body_index = hammer_body_index
        self._n_hat = n_hat / np.linalg.norm(n_hat)

        # Abstract input: ContactResults
        self.DeclareAbstractInputPort(
            "contact_results",
            AbstractValue.Make(ContactResults())
        )

        # Scalar output: F_meas
        self.DeclareVectorOutputPort(
            "F_meas", BasicVector(1),
            self.CalcOutput
        )

    def CalcOutput(self, context, output):
        contact_results = self.get_input_port(0).Eval(context)
        F_W = np.zeros(3)

        for i in range(contact_results.num_point_pair_contacts()):
            info = contact_results.point_pair_contact_info(i)
            f_Bc_W = np.array(info.contact_force())
            bodyA = info.bodyA_index()
            bodyB = info.bodyB_index()

            if bodyA == self._hammer_body_index:
                F_W -= f_Bc_W    # equal/opposite on A
            if bodyB == self._hammer_body_index:
                F_W += f_Bc_W

        F_meas = float(self._n_hat.dot(F_W))
        output.SetAtIndex(0, F_meas)


class HitSequenceAdmittance(LeafSystem):
    """
    3-phase controller with 1D admittance + joint-space posture spring:

      Phase 1 (approach): follow joint-space trajectory traj(t) for ALL joints.
      Phase 2 (hit): 1D admittance along n_hat using ALL joints to realize
                     the motion; also apply posture spring toward q_prehit.
      Phase 3 (retract): same as Phase 2 but with F_des_eff = 0 so
                         the virtual spring K_a pulls s → 0 (retract hammer).

    State: [s, s_dot] = [displacement along n_hat, velocity along n_hat]
    Inputs:
      0: F_meas (scalar)    - measured force along n_hat
      1: q_meas (7-vector)  - measured iiwa joint positions
    Output:
      q_cmd (7-vector)      - joint position command
    """
    def __init__(self, traj_approach, t_hit_start, hit_duration,
                 q_prehit, J_pinv, n_hat,
                 F_des,
                 M_a=1.0, D_a=40.0, K_a=200.0,
                 K_null=5.0,
                 retract_duration=1.5):
        super().__init__()

        self._traj = traj_approach
        self._t_hit_start = float(t_hit_start)
        self._hit_duration = float(hit_duration)

        self._q_prehit = q_prehit.copy().reshape(-1)   # 7
        self._J_pinv = J_pinv.copy()                   # (7, 3)
        self._n_hat = n_hat / np.linalg.norm(n_hat)

        self._F_des_hit = float(F_des)
        self._M_a = float(M_a)
        self._D_a = float(D_a)
        self._K_a = float(K_a)     # used mainly in retract
        self._K_null = float(K_null)  # posture spring gain
        self._retract_duration = float(retract_duration)

        # Continuous state: [s, s_dot]
        self.DeclareContinuousState(2)

        # Input 0: measured force along n_hat
        self.DeclareVectorInputPort("F_meas", BasicVector(1))
        # Input 1: measured joint positions (q_meas)
        self.DeclareVectorInputPort("q_meas", BasicVector(len(self._q_prehit)))

        # Output: joint position command q_cmd
        self.DeclareVectorOutputPort(
            "q_cmd", BasicVector(len(self._q_prehit)),
            self.CalcOutput
        )
                # Output 1: internal admittance state [s, s_dot]
        self.DeclareVectorOutputPort(
            "admittance_state", BasicVector(2),
            self.CalcStateOutput
        )

    def DoCalcTimeDerivatives(self, context, derivatives):
        t = context.get_time()
        x = context.get_continuous_state_vector().CopyToVector()
        s = x[0]
        v = x[1]

        if t < self._t_hit_start:
            # Phase 1: approach – keep admittance state frozen
            ds = 0.0
            dv = 0.0

        else:
            tau = t - self._t_hit_start
            F_meas = self.get_input_port(0).Eval(context)[0]

            if tau <= self._hit_duration:
                # Phase 2: HIT – true force control
                F_des_eff = self._F_des_hit
                F_err = F_des_eff - F_meas

            else:
                # Phase 3: RETRACT – ignore measured force, just spring back to s = 0
                F_err = 0.0   # <--- KEY CHANGE: no F_meas in retract

            ds = v
            dv = (F_err - self._D_a * v - self._K_a * s) / self._M_a

        der = derivatives.get_mutable_vector()
        der[0] = ds
        der[1] = dv

    def CalcOutput(self, context, output):
        t = context.get_time()
        x = context.get_continuous_state_vector().CopyToVector()
        s = x[0]

        if t < self._t_hit_start:
            # Phase 1: approach
            q_cmd = self._traj.value(t).flatten()

        else:
            tau = t - self._t_hit_start

            if tau <= self._hit_duration:
                # Phase 2: HIT – admittance + posture spring
                dx_W = self._n_hat * s
                dq   = self._J_pinv @ dx_W

                q_meas = self.get_input_port(1).Eval(context)
                q_err  = self._q_prehit - q_meas
                q_null = self._K_null * q_err

                q_cmd = self._q_prehit + dq + q_null

            elif tau <= self._hit_duration + self._retract_duration:
                # Phase 3: RETRACT – s is being driven toward 0, still use posture spring
                dx_W = self._n_hat * s
                dq   = self._J_pinv @ dx_W

                q_meas = self.get_input_port(1).Eval(context)
                q_err  = self._q_prehit - q_meas
                q_null = self._K_null * q_err

                q_cmd = self._q_prehit + dq + q_null

            else:
                # Phase 4: DONE – lock exactly at pre-hit pose
                q_cmd = self._q_prehit

        output.SetFromVector(q_cmd)

    def CalcStateOutput(self, context, output):
        x = context.get_continuous_state_vector().CopyToVector()
        output.SetFromVector(x)  # [s, s_dot]

In [61]:
scenario = LoadScenario(data=scenario_string)
station = MakeHardwareStation(scenario, meshcat=meshcat)
builder = DiagramBuilder()
builder.AddSystem(station)
plant = station.GetSubsystemByName("plant")

temp_context = station.CreateDefaultContext()
temp_plant_context = plant.GetMyContextFromRoot(temp_context)

# # Build mole joints dictionary
mole_joints = {}
for i in range(3):
    for j in range(3):
        model_name = f"mole_{i}_{j}"
        instance = plant.GetModelInstanceByName(model_name)
        joint = plant.GetJointByName("mole_slider", instance)
        mole_joints[(i, j)] = joint

# Add mole controller
controller = builder.AddSystem(
    PoppingMoleController(
        plant,
        mole_joints,
        rise_height=0.09,
        rest_height=0.0,
        min_up=3,
        max_up=3
    )
)

# Connect plant state → controller
builder.Connect(
    station.GetOutputPort("state"),
    controller.state_in
)

# Connect each mole controller output → station actuation port
for i in range(3):
    for j in range(3):
        builder.Connect(
            controller.mole_out[(i, j)],
            station.GetInputPort(f"mole_{i}_{j}_actuation")
        )

In [62]:
X_WGinitial = plant.EvalBodyPoseInWorld(temp_plant_context, plant.GetBodyByName("iiwa_link_7"))
print(X_WGinitial)

# Get hammer head frame
hammer = plant.GetModelInstanceByName("hammer")
hammer_face_frame = plant.GetFrameByName("hammer_face", hammer)
hammer_body = plant.GetBodyByName("hammer_link", hammer)
hammer_body_index = hammer_body.index()
X_WHammer = hammer_face_frame.CalcPoseInWorld(temp_plant_context)

# Get mole pose
mole_instance = plant.GetModelInstanceByName("mole_1_1")

mole_body = plant.GetBodyByName("mole", mole_instance)
X_WMole = plant.EvalBodyPoseInWorld(temp_plant_context, mole_body)

# # get pose of iiwa link 7 in hammer face frame
iiwa = plant.GetModelInstanceByName("iiwa")
l7_frame = plant.GetFrameByName("iiwa_link_7", iiwa)
X_HL7 = plant.CalcRelativeTransform(temp_plant_context, hammer_face_frame, l7_frame)

# get pre-hit pose frame
X_WH_prehit, X_WL7_prehit = get_mole_prehit_pose(X_WMole, X_HL7)
print("X_WH_prehit",X_WH_prehit)
print("X_WL7_prehit",X_WL7_prehit)

# visualize extra prehit frames
AddMeshcatTriad(
    meshcat,
    path="hammer_prehit_pose_triad",  
    length=0.1,
    radius=0.005,
    X_PT=X_WH_prehit,                 
)
AddMeshcatTriad(
    meshcat,
    path="iiwa_prehit_pose_triad",  
    length=0.1,
    radius=0.005,
    X_PT=X_WL7_prehit,                 
)

# Get initial positions of the iiwa joints
iiwa_q0 = plant.GetPositions(temp_plant_context, iiwa)
print("iiwa q0", iiwa_q0)

# solve ik for goal joint config
q_goal, optimal = solve_ik(plant, temp_plant_context, X_WH_prehit, q_guess=iiwa_q0,
                    pos_tol=1e-6, theta_bound=1e-3)
print(q_goal, optimal)

# linear joint space position trajectory
path = [iiwa_q0, q_goal[0:7,]]
traj = make_joint_space_position_trajectory(path)
# iiwa_src = builder.AddSystem(TrajectorySource(traj))
# builder.Connect(iiwa_src.get_output_port(), station.GetInputPort("iiwa.position"))

RigidTransform(
  R=RotationMatrix([
    [-0.000773199921913291, 0.999999682931835, 0.00019052063137842191],
    [0.9709578572896667, 0.0007963267107333862, -0.23924925335563643],
    [-0.23924932921398254, 2.0149982078937825e-16, -0.9709581651495918],
  ]),
  p=[0.0003536363619349402, -0.444084375221239, 0.7667078137695524],
)
X_WH_prehit RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, -1.0, 0.0],
    [0.0, 0.0, -1.0],
  ]),
  p=[0.0, -0.75, 0.1325],
)
X_WL7_prehit RigidTransform(
  R=RotationMatrix([
    [0.9999999999932544, -4.2028025922252773e-17, -3.6732051029997326e-06],
    [4.1920324360180517e-17, -1.0000000000000009, -8.678676966084764e-17],
    [-3.6732051039156666e-06, -8.673238247311152e-17, -0.9999999999932563],
  ]),
  p=[0.30000049588066513, -0.75, 0.26749889803755844],
)
iiwa q0 [-1.57  0.1   0.   -1.2   0.    1.6   0.  ]
[-1.18758980e+00  1.55943561e+00 -2.09750219e-01 -6.72739509e-07
  2.09672534e-01  1.56692215e+00  1.93421608e+00  0.00000000e+00


In [63]:
# === Hit direction and hit parameters ===
# World positions
p_mole_W = X_WMole.translation()          # mole center in world
p_hammer_prehit_W = X_WH_prehit.translation()  # hammer pre-hit position in world

# Direction from hammer pre-hit -> mole
n_hat = p_mole_W - p_hammer_prehit_W      # vector pointing TOWARDS mole
n_hat = n_hat / np.linalg.norm(n_hat)

print("p_mole_W:", p_mole_W)
print("p_hammer_prehit_W:", p_hammer_prehit_W)
print("n_hat (hammer -> mole):", n_hat)

# Desired hit parameters
F_des = 200.0           # [N] desired normal force (tune)
v_pre = 1               # [m/s] initial velocity along n_hat at contact (tune)
hit_duration = 0.5      # [s] time to maintain the hit (tune)

p_mole_W: [ 0.     -0.75    0.0125]
p_hammer_prehit_W: [ 0.     -0.75    0.1325]
n_hat (hammer -> mole): [ 0.  0. -1.]


In [64]:
# === Compute Jacobian at pre-hit pose ===
J_context = plant.CreateDefaultContext()
plant.SetPositions(J_context, iiwa, q_goal[0:7,])

p_HQ_H = np.zeros(3)

# Full translational Jacobian: v_WQ = Jv_WQ * v
Jv_WQ_full = plant.CalcJacobianTranslationalVelocity(
    J_context,
    JacobianWrtVariable.kV,
    hammer_face_frame,   # hammer face frame
    p_HQ_H,              # origin of hammer face
    plant.world_frame(), # measure in world
    plant.world_frame(), # express in world
)
print("Jv_WQ_full shape:", Jv_WQ_full.shape)  # (3, 7)

# Extract iiwa portion of Jacobian
Jv_WQ = Jv_WQ_full[:, :7]
print("IIWA Jacobian shape:", Jv_WQ.shape)   # (3, 7)

# Full pseudoinverse: dx (3) -> dq (7)
J_pinv = np.linalg.pinv(Jv_WQ)      # shape (7, 3)
print("J_pinv shape:", J_pinv.shape)

Jv_WQ shape: (3, 7)
IIWA Jacobian shape: (3, 7)
J_pinv shape: (7, 3)


In [65]:
# === Add force and admittance systems, then build diagram ===
# 1. Contact force extraction

force_sys = builder.AddSystem(
    HammerContactForce(plant, hammer_body_index, n_hat)
)
builder.Connect(
    station.get_output_port(18),    # 'contact_results'
    force_sys.get_input_port(0)
)

# 1b. NEW: log the scalar normal force F_meas
force_logger = LogVectorOutput(
    force_sys.get_output_port(0),   # F_meas
    builder
)

# 2. Hit + admittance + retract controller (full J + posture spring)
t_hit_start = traj.end_time()   # Time when approach trajectory reaches pre-hit pose

hit_ctrl = builder.AddSystem(
    HitSequenceAdmittance(
        traj_approach=traj,
        t_hit_start=t_hit_start,
        hit_duration=hit_duration,
        q_prehit=q_goal[0:7,],
        J_pinv=J_pinv,
        n_hat=n_hat,
        F_des=F_des,
        M_a=1.0,
        D_a=40.0,
        K_a=200.0,   # mainly for retract
        K_null=5.0,  # posture spring gain; tune 3–10
        retract_duration=1.5,
    )
)

# Connect measured force into admittance (input 0)
builder.Connect(
    force_sys.get_output_port(0),
    hit_ctrl.get_input_port(0)
)

# Connect measured iiwa joint positions into posture spring (input 1)
# From your port list, index 3 is 'iiwa.position_measured'
builder.Connect(
    station.get_output_port(3),
    hit_ctrl.get_input_port(1)
)

# 3. Controller output goes to iiwa.position command
builder.Connect(
    hit_ctrl.get_output_port(0),
    station.GetInputPort("iiwa.position")
)

# Log admittance internal state [s, s_dot]
admittance_logger = LogVectorOutput(
    hit_ctrl.get_output_port(1),   # "admittance_state"
    builder
)

# Log measured joint positions
q_logger = LogVectorOutput(
    station.get_output_port(3),  # 'iiwa.position_measured'
    builder
)

# Log estimated joint velocities
v_logger = LogVectorOutput(
    station.get_output_port(4),  # 'iiwa.velocity_estimated'
    builder
)

diagram = builder.Build()

In [66]:
# Run simulation
simulator = Simulator(diagram)

ctx = simulator.get_mutable_context()
diagram.ForcedPublish(ctx)

meshcat.StartRecording()

# simulator.AdvanceTo(traj.end_time() if running_as_notebook else 5)
retract_duration = 20
t_final = t_hit_start + hit_duration + retract_duration
simulator.AdvanceTo(t_final)

meshcat.StopRecording()
meshcat.PublishRecording()

In [67]:
# meshcat.Delete()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2500cebb-ccf6-40e5-a2c5-6c1beb3b7769' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>